# Results in tables

In [1]:
import pandas as pd
import os
from collections import defaultdict
from IPython.display import display

In [2]:
results_folder = "results"
model_types = ["cnn14_logstft", "cnn14_logmel"]
method_names = ["saliency", "gradcam", "lime", "shap"]
exp_types_mapping = {
    "cnn14_logstft": ["clean"],
    "cnn14_logmel": ["clean", "white", "room", "horse"]
}
folder_for_tables = "csvs"

In [3]:
csv_files = []
for model in model_types:
    for method in method_names:
        for exp in exp_types_mapping[model]:
            dir_path = os.path.join(results_folder,
                                    model,
                                    f"{method}_{exp}",
                                    folder_for_tables)
            if not os.path.isdir(dir_path):
                continue
            for fname in os.listdir(dir_path):
                if fname.endswith(".csv"):
                    csv_files.append(os.path.join(dir_path, fname))

In [4]:
experiment_results = {m: {} for m in model_types}
for fpath in csv_files:
    parts = fpath.split(os.sep)
    model_type = parts[parts.index(results_folder) + 1]
    method, exp = parts[parts.index(results_folder) + 2].split("_", 1)
    exp_name = f"{method}_{exp}"
    mask_name = os.path.splitext(parts[-1])[0]
    
    df = pd.read_csv(fpath)
    metrics = df.columns.difference(["sample", "is_correct"])
    means = df[metrics].mean()
    
    experiment_results[model_type].setdefault(exp_name, {})[mask_name] = means

In [5]:
for model, exps in experiment_results.items():
    for exp_name, mask_dict in exps.items():
        experiment_results[model][exp_name] = pd.DataFrame(mask_dict).T

In [6]:
experiment_results_true  = {m: {} for m in model_types}
experiment_results_false = {m: {} for m in model_types}

for fpath in csv_files:
    parts = fpath.split(os.sep)
    model_type  = parts[parts.index(results_folder) + 1]
    method, exp = parts[parts.index(results_folder) + 2].split("_", 1)
    exp_name = f"{method}_{exp}"
    mask_name = os.path.splitext(parts[-1])[0]
    
    df = pd.read_csv(fpath)
    metrics = df.columns.difference(["sample", "is_correct"])
    
    means_t = df[df["is_correct"] == True][metrics].mean()
    means_f = df[df["is_correct"] == False][metrics].mean()
    
    experiment_results_true[model_type].setdefault(exp_name, {})[mask_name]  = means_t
    experiment_results_false[model_type].setdefault(exp_name, {})[mask_name] = means_f

for m in model_types:
    for exp_name, mask_dict in experiment_results_true[m].items():
        experiment_results_true[m][exp_name] = pd.DataFrame(mask_dict).T
    for exp_name, mask_dict in experiment_results_false[m].items():
        experiment_results_false[m][exp_name] = pd.DataFrame(mask_dict).T

In [7]:
mask_experiment_results = {m: {} for m in model_types}
for fpath in csv_files:
    parts = fpath.split(os.sep)
    model_type = parts[parts.index(results_folder) + 1]
    method, exp = parts[parts.index(results_folder) + 2].split("_", 1)
    mask_name = os.path.splitext(parts[-1])[0]
    
    df = pd.read_csv(fpath)
    metrics = df.columns.difference(["sample", "is_correct"])
    means = df[metrics].mean()
    
    mask_experiment_results[model_type]\
      .setdefault(exp, {})\
      .setdefault(mask_name, {})[method] = means

for model, exps in mask_experiment_results.items():
    for exp_type, masks in exps.items():
        for mask_name, methods_dict in masks.items():
            mask_experiment_results[model][exp_type][mask_name] = \
                pd.DataFrame(methods_dict).T

In [8]:
column_order    = ["FF", "AI", "AG", "FidIn", "SPS", "AD", "COMP"]
invert_metrics  = {"AD", "COMP"}
diff_color_cols = {"SPS", "COMP"}

In [9]:
def highlight_extremes(col: pd.Series):
    is_invert = col.name in invert_metrics
    use_blue  = col.name in diff_color_cols
    mx, mn    = col.max(), col.min()
    styles    = []
    for v in col:
        if use_blue:
            if is_invert:
                if v == mn: styles.append("background-color: lightblue; font-weight: bold")
                elif v == mx: styles.append("background-color: lightsteelblue; font-weight: bold")
                else: styles.append("")
            else:
                if v == mx: styles.append("background-color: lightblue; font-weight: bold")
                elif v == mn: styles.append("background-color: lightsteelblue; font-weight: bold")
                else: styles.append("")
        else:
            if is_invert:
                if v == mn: styles.append("background-color: lightgreen; font-weight: bold")
                elif v == mx: styles.append("background-color: lightcoral; font-weight: bold")
                else: styles.append("")
            else:
                if v == mx: styles.append("background-color: lightgreen; font-weight: bold")
                elif v == mn: styles.append("background-color: lightcoral; font-weight: bold")
                else: styles.append("")
    return styles

In [10]:
def display_experiment_results(results):
    def _show(df, title=None):
        df = df.reindex(columns=[c for c in column_order if c in df.columns])
        styled = df.style.apply(highlight_extremes, axis=0)
        if title:
            print(f"\n--- {title} ---")
        display(styled)
    
    if isinstance(results, pd.DataFrame):
        _show(results)
        return
    
    if all(isinstance(v, pd.DataFrame) for v in results.values()):
        for name, df in results.items():
            _show(df, title=name)
        return
    
    for model, exps in results.items():
        print(f"\n===== MODEL: {model} =====")
        for name, df in exps.items():
            _show(df, title=name)

In [11]:
def display_summary_table(df, title=None):
    order = [m for m in ["saliency","gradcam","lime","shap"] if (m in df.index) or ("method" in df.columns and m in df["method"].tolist())]
    df = df.set_index("method")
    df = df.loc[order]
    df = df.reindex(columns=[c for c in column_order if c in df.columns])
    styled = df.style.apply(highlight_extremes, axis=0)
    if title:
        print(f"\n=== {title} ===")
    display(styled)

## Mel model - clean

In [12]:
display_experiment_results(experiment_results["cnn14_logmel"]["saliency_clean"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.923509,17.000000,0.010820,0.660000,0.173750,34.228729,3.364614
topK_30_pos,0.924342,0.000000,0.000000,0.015000,0.700006,98.380550,9.171495
minmax_pos,0.765816,0.250000,0.049572,0.087500,0.789568,94.036667,9.025774
topK_5_pos,0.760572,0.000000,0.000000,0.022500,0.949973,97.641902,7.380256
pos_thresh_50,0.034041,0.250000,0.173731,0.017500,0.998818,97.718493,3.126737
topK_50,0.925488,0.000000,0.000000,0.020000,0.500000,98.354752,9.682342
pos_thresh_25,0.266565,0.500000,0.321346,0.025000,0.990407,97.161012,5.499278
bin,0.925405,0.000000,0.000000,0.020000,0.501437,98.364178,9.679418
minmax,0.865400,3.750000,3.145995,0.270000,0.038238,77.512300,10.372001
topK_5,0.760572,0.000000,0.000000,0.022500,0.949973,97.641902,7.380256


In [13]:
display_experiment_results(experiment_results["cnn14_logmel"]["gradcam_clean"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.620931,11.750000,0.986515,0.652500,0.376861,38.971917,7.311441
topK_30_pos,0.362777,8.000000,0.610826,0.445000,0.594143,60.972215,7.798626
minmax_pos,0.431486,0.750000,0.675307,0.362500,0.396158,70.565552,9.305825
topK_5_pos,0.079183,3.250000,0.000789,0.140000,0.889483,88.374592,6.940699
pos_thresh_50,0.366132,4.250000,1.610246,0.462500,0.539787,60.367838,8.580185
topK_50,0.525585,4.500000,2.390889,0.547500,0.499100,50.501074,9.684130
pos_thresh_25,0.599464,12.250000,2.903764,0.685000,0.347608,37.228447,7.528186
bin,0.718901,19.500000,2.312055,0.782500,0.195505,24.363625,4.918478
minmax,0.513961,1.250000,0.771900,0.392500,0.335331,66.672894,10.150634
topK_5,0.030628,0.000000,0.000000,0.095000,0.948803,93.808333,7.403008


In [14]:
display_experiment_results(experiment_results["cnn14_logmel"]["lime_clean"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.827086,13.500000,5.598555,0.750000,0.461311,30.218063,9.410670
topK_30_pos,0.623726,10.000000,4.870080,0.542500,0.674179,49.493281,9.251675
minmax_pos,0.757147,4.000000,1.739817,0.437500,0.547565,60.697763,9.736422
topK_5_pos,0.159315,2.000000,1.222972,0.305000,0.913795,72.973278,7.872777
pos_thresh_50,0.486153,3.000000,1.786661,0.472500,0.748973,57.984101,8.786603
topK_50,0.825345,13.750000,6.322732,0.740000,0.476548,31.117691,9.727269
pos_thresh_25,0.802991,9.750000,4.796789,0.700000,0.561653,35.022268,9.408597
bin,0.899183,28.750000,12.359402,0.932500,0.200351,8.067020,9.534232
minmax,0.788818,4.750000,2.500606,0.482500,0.392213,56.327209,10.072025
topK_5,0.159315,2.000000,1.222972,0.305000,0.913795,72.973278,7.872777


In [15]:
display_experiment_results(experiment_results["cnn14_logmel"]["shap_clean"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.927128,0.000000,0.000000,0.017500,0.500000,98.489136,9.682342
topK_30_pos,0.920323,0.000000,0.000000,0.015000,0.700006,98.376693,9.171495
minmax_pos,0.742941,0.000000,0.000000,0.017500,0.730384,98.140806,9.325663
topK_5_pos,0.644693,0.000000,0.000000,0.027500,0.949973,97.529152,7.380256
pos_thresh_50,0.059368,0.250000,0.190206,0.025000,0.996374,97.753463,4.424030
topK_50,0.927128,0.000000,0.000000,0.017500,0.500000,98.489136,9.682342
pos_thresh_25,0.514186,0.000000,0.000000,0.020000,0.955756,97.999591,7.109132
bin,0.927023,0.000000,0.000000,0.015000,0.488779,98.448432,9.704492
minmax,0.896360,0.000000,0.000000,0.080000,0.069193,93.939058,10.366691
topK_5,0.644693,0.000000,0.000000,0.027500,0.949973,97.529152,7.380256


## STFT model - clean

In [16]:
display_experiment_results(experiment_results["cnn14_logstft"]["saliency_clean"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.804243,21.250000,0.005357,0.515000,0.246247,48.376074,5.221556
topK_30_pos,0.775971,1.000000,0.695137,0.030000,0.699991,96.617813,10.091335
minmax_pos,0.793905,1.250000,0.374701,0.045000,0.783134,94.492255,9.997833
topK_5_pos,0.617889,0.750000,0.579243,0.030000,0.949986,96.632662,8.299783
pos_thresh_50,0.031167,0.500000,0.253879,0.030000,0.999009,96.453913,3.677172
topK_50,0.791540,0.500000,0.431110,0.027500,0.499994,96.391078,10.602145
pos_thresh_25,0.219041,1.250000,0.177886,0.040000,0.991120,95.720335,6.327378
bin,0.791281,0.500000,0.430915,0.027500,0.500274,96.420557,10.601551
minmax,0.788165,10.000000,6.286535,0.342500,0.038138,66.196693,11.292008
topK_5,0.617889,0.750000,0.579243,0.030000,0.949986,96.632662,8.299783


In [17]:
display_experiment_results(experiment_results["cnn14_logstft"]["gradcam_clean"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.724579,24.750000,0.217910,0.702500,0.224974,31.075000,4.771012
topK_30_pos,0.572529,16.250000,0.219069,0.517500,0.442721,50.726126,6.382848
minmax_pos,0.320230,0.500000,0.087757,0.100000,0.610100,90.744003,9.346533
topK_5_pos,0.191031,6.250000,0.071148,0.157500,0.835978,84.384347,7.304013
pos_thresh_50,0.266586,0.250000,0.207937,0.117500,0.762384,89.263126,8.492369
topK_50,0.585838,4.000000,1.152864,0.437500,0.499938,59.861858,10.602257
pos_thresh_25,0.398511,2.250000,0.973528,0.267500,0.600928,73.924376,9.142811
bin,0.474595,7.500000,1.070762,0.497500,0.451437,54.120088,8.475080
minmax,0.567456,1.750000,0.520995,0.262500,0.295466,75.694391,11.139094
topK_5,0.114764,0.250000,0.070154,0.040000,0.949974,95.877295,8.300039


In [18]:
display_experiment_results(experiment_results["cnn14_logstft"]["lime_clean"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.765314,16.750000,8.681377,0.452500,0.472411,55.331106,10.164076
topK_30_pos,0.694155,10.000000,5.252163,0.337500,0.691593,67.192280,10.118696
minmax_pos,0.633585,1.000000,0.381902,0.180000,0.621340,84.303404,10.527688
topK_5_pos,0.250140,0.750000,0.135246,0.135000,0.936524,89.718892,8.527748
pos_thresh_50,0.431767,0.750000,0.356323,0.182500,0.868335,84.286786,9.100906
topK_50,0.764448,15.500000,9.093660,0.427500,0.493322,57.746470,10.615335
pos_thresh_25,0.676561,5.750000,2.650058,0.335000,0.697021,68.592485,9.986101
bin,0.790285,26.000000,14.205455,0.685000,0.272667,32.028679,10.931619
minmax,0.685044,3.000000,1.561553,0.237500,0.311617,77.522342,11.105058
topK_5,0.250140,0.750000,0.135246,0.135000,0.936524,89.718892,8.527748


In [19]:
display_experiment_results(experiment_results["cnn14_logstft"]["shap_clean"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.802137,3.750000,0.377866,0.117500,0.448744,88.001432,9.515425
topK_30_pos,0.763906,0.500000,0.387102,0.027500,0.699991,97.378508,10.091335
minmax_pos,0.455331,0.250000,0.026532,0.020000,0.733023,96.696681,10.238195
topK_5_pos,0.444995,0.250000,0.030072,0.017500,0.949986,96.534742,8.299783
pos_thresh_50,0.042832,0.750000,0.169498,0.010000,0.997583,97.548774,4.935396
topK_50,0.797198,0.500000,0.377006,0.020000,0.499994,97.530510,10.602145
pos_thresh_25,0.325835,0.000000,0.000000,0.020000,0.963614,97.014994,7.850671
bin,0.796341,0.500000,0.376295,0.020000,0.491457,97.488113,10.618963
minmax,0.758164,1.250000,0.592583,0.085000,0.072735,92.499200,11.285742
topK_5,0.444995,0.250000,0.030072,0.017500,0.949986,96.534742,8.299783


## Mel model clean - correct

In [20]:
display_experiment_results(experiment_results_true["cnn14_logmel"]["saliency_clean"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.945407,17.073171,0.011396,0.680217,0.163957,32.330323,3.174969
topK_30_pos,0.945672,0.000000,0.000000,0.016260,0.700006,98.377567,9.171495
minmax_pos,0.779311,0.000000,0.000000,0.092141,0.788313,94.163711,9.036151
topK_5_pos,0.772949,0.000000,0.000000,0.024390,0.949973,97.640883,7.380256
pos_thresh_50,0.029163,0.271003,0.188326,0.018970,0.998817,97.861775,3.133157
topK_50,0.946665,0.000000,0.000000,0.021680,0.500000,98.339428,9.682342
pos_thresh_25,0.259918,0.271003,0.169656,0.021680,0.990384,97.379375,5.511346
bin,0.946579,0.000000,0.000000,0.021680,0.501572,98.350047,9.679155
minmax,0.882767,4.065041,3.410292,0.279133,0.038666,76.540316,10.371953
topK_5,0.772949,0.000000,0.000000,0.024390,0.949973,97.640883,7.380256


In [21]:
display_experiment_results(experiment_results_true["cnn14_logmel"]["gradcam_clean"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.630246,10.569106,0.885289,0.666667,0.384207,38.209700,7.453223
topK_30_pos,0.362149,7.046070,0.438331,0.447154,0.604240,61.349777,7.931774
minmax_pos,0.442164,0.542005,0.497571,0.379404,0.389012,69.407910,9.366113
topK_5_pos,0.077364,2.710027,0.000678,0.138211,0.892246,88.718802,6.961431
pos_thresh_50,0.376808,4.065041,1.477375,0.479675,0.535387,58.944749,8.656051
topK_50,0.534547,4.336043,2.269330,0.577236,0.499126,48.482414,9.684078
pos_thresh_25,0.619630,12.466125,2.900091,0.712737,0.339949,35.056717,7.534071
bin,0.743637,19.241192,2.057722,0.802168,0.185910,22.318007,4.848208
minmax,0.523824,1.084011,0.568076,0.411924,0.336639,65.374030,10.149080
topK_5,0.028273,0.000000,0.000000,0.094851,0.948832,93.922137,7.402466


In [22]:
display_experiment_results(experiment_results_true["cnn14_logmel"]["lime_clean"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.847274,13.279133,5.791467,0.769648,0.463530,28.938404,9.463903
topK_30_pos,0.632108,9.485095,4.781647,0.555556,0.673927,48.656539,9.252500
minmax_pos,0.772356,3.794038,1.600055,0.460705,0.543287,59.272476,9.750867
topK_5_pos,0.145929,1.355014,0.881449,0.311653,0.912932,72.477098,7.881742
pos_thresh_50,0.492048,2.710027,1.588524,0.490515,0.744019,56.764825,8.818588
topK_50,0.845376,13.821138,6.459969,0.764228,0.476298,29.388786,9.727778
pos_thresh_25,0.819008,9.485095,4.675122,0.731707,0.555752,32.783006,9.432027
bin,0.920931,28.726287,12.686897,0.953930,0.195358,6.189071,9.547627
minmax,0.805307,4.607046,2.325402,0.504065,0.394354,54.714965,10.071379
topK_5,0.145929,1.355014,0.881449,0.311653,0.912932,72.477098,7.881742


In [23]:
display_experiment_results(experiment_results_true["cnn14_logmel"]["shap_clean"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.948405,0.000000,0.000000,0.018970,0.500000,98.487189,9.682342
topK_30_pos,0.942526,0.000000,0.000000,0.016260,0.700006,98.375260,9.171495
minmax_pos,0.756791,0.000000,0.000000,0.018970,0.730700,98.266224,9.324414
topK_5_pos,0.660258,0.000000,0.000000,0.027100,0.949973,97.559801,7.380256
pos_thresh_50,0.056379,0.000000,0.000000,0.021680,0.996380,97.994332,4.427683
topK_50,0.948405,0.000000,0.000000,0.018970,0.500000,98.487189,9.682342
pos_thresh_25,0.524006,0.000000,0.000000,0.018970,0.955952,98.076177,7.102215
bin,0.948274,0.000000,0.000000,0.016260,0.488946,98.450221,9.704169
minmax,0.916186,0.000000,0.000000,0.084011,0.069038,93.863099,10.366718
topK_5,0.660258,0.000000,0.000000,0.027100,0.949973,97.559801,7.380256


## Mel model clean - incorrect

In [24]:
display_experiment_results(experiment_results_false["cnn14_logmel"]["saliency_clean"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.662846,16.129032,0.003964,0.419355,0.290323,56.825893,5.622005
topK_30_pos,0.670454,0.000000,0.000000,0.000000,0.700006,98.416065,9.171495
minmax_pos,0.605179,3.225806,0.639638,0.032258,0.804512,92.524422,8.902249
topK_5_pos,0.613254,0.000000,0.000000,0.000000,0.949973,97.654032,7.380256
pos_thresh_50,0.092098,0.000000,0.000000,0.000000,0.998837,96.012966,3.050318
topK_50,0.673416,0.000000,0.000000,0.000000,0.500000,98.537152,9.682342
pos_thresh_25,0.345686,3.225806,2.126949,0.064516,0.990689,94.561785,5.355628
bin,0.673358,0.000000,0.000000,0.000000,0.499835,98.532379,9.682547
minmax,0.658679,0.000000,0.000000,0.161290,0.033137,89.082037,10.372571
topK_5,0.613254,0.000000,0.000000,0.000000,0.949973,97.654032,7.380256


In [25]:
display_experiment_results(experiment_results_false["cnn14_logmel"]["gradcam_clean"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.510053,25.806452,2.191429,0.483871,0.289428,48.044750,5.623778
topK_30_pos,0.370261,19.354839,2.664075,0.419355,0.473962,56.478008,6.213730
minmax_pos,0.304390,3.225806,2.790927,0.161290,0.481220,84.345221,8.588200
topK_5_pos,0.100836,9.677419,0.002109,0.161290,0.856589,84.277379,6.693920
pos_thresh_50,0.239048,6.451613,3.191839,0.258065,0.592162,77.307186,7.677133
topK_50,0.418908,6.451613,3.837831,0.193548,0.498788,74.529634,9.684749
pos_thresh_25,0.359428,9.677419,2.947487,0.354839,0.438772,63.079039,7.458135
bin,0.424467,22.580645,5.339440,0.548387,0.309716,48.713074,5.754914
minmax,0.396560,3.225806,3.198057,0.161290,0.319760,82.133567,10.169128
topK_5,0.058653,0.000000,0.000000,0.096774,0.948455,92.453693,7.409452


In [26]:
display_experiment_results(experiment_results_false["cnn14_logmel"]["lime_clean"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.586783,16.129032,3.302271,0.516129,0.434898,45.450130,8.777023
topK_30_pos,0.523954,16.129032,5.922719,0.387097,0.677181,59.453209,9.241862
minmax_pos,0.576111,6.451613,3.403436,0.161290,0.598479,77.663271,9.564484
topK_5_pos,0.318655,9.677419,5.288197,0.225806,0.924064,78.879420,7.766068
pos_thresh_50,0.415989,6.451613,4.145134,0.258065,0.807937,72.497425,8.405885
topK_50,0.586913,12.903226,4.689164,0.451613,0.479526,51.697237,9.721214
pos_thresh_25,0.612332,12.903226,6.245024,0.322581,0.631892,61.676711,9.129704
bin,0.640306,29.032258,8.461155,0.677419,0.259775,30.420668,9.374790
minmax,0.592543,6.451613,4.586105,0.225806,0.366728,75.518108,10.079711
topK_5,0.318655,9.677419,5.288197,0.225806,0.924064,78.879420,7.766068


In [27]:
display_experiment_results(experiment_results_false["cnn14_logmel"]["shap_clean"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.673863,0.000000,0.000000,0.000000,0.500000,98.512308,9.682342
topK_30_pos,0.656046,0.000000,0.000000,0.000000,0.700006,98.393752,9.171495
minmax_pos,0.578087,0.000000,0.000000,0.000000,0.726615,96.647926,9.340534
topK_5_pos,0.459420,0.000000,0.000000,0.032258,0.949973,97.164324,7.380256
pos_thresh_50,0.094950,3.225806,2.454268,0.064516,0.996303,94.886344,4.380559
topK_50,0.673863,0.000000,0.000000,0.000000,0.500000,98.512308,9.682342
pos_thresh_25,0.397294,0.000000,0.000000,0.032258,0.953419,97.087980,7.191464
bin,0.674068,0.000000,0.000000,0.000000,0.486790,98.427143,9.708335
minmax,0.660365,0.000000,0.000000,0.032258,0.071041,94.843223,10.366376
topK_5,0.459420,0.000000,0.000000,0.032258,0.949973,97.164324,7.380256


## STFT model clean - correct

In [28]:
display_experiment_results(experiment_results_true["cnn14_logstft"]["saliency_clean"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.866403,21.153846,0.006039,0.519231,0.245189,47.934814,5.199129
topK_30_pos,0.838052,0.320513,0.019933,0.028846,0.699991,96.769077,10.091335
minmax_pos,0.851453,0.320513,0.038429,0.035256,0.781555,95.605697,10.009087
topK_5_pos,0.647590,0.000000,0.000000,0.028846,0.949986,96.805100,8.299783
pos_thresh_50,0.024769,0.320513,0.133165,0.028846,0.999013,96.935649,3.706453
topK_50,0.853016,0.000000,0.000000,0.028846,0.499994,96.267474,10.602145
pos_thresh_25,0.206891,0.641026,0.037393,0.044872,0.991009,95.848175,6.354969
bin,0.852802,0.000000,0.000000,0.028846,0.500330,96.315157,10.601439
minmax,0.848827,8.333333,5.478854,0.371795,0.038374,65.418301,11.291996
topK_5,0.647590,0.000000,0.000000,0.028846,0.949986,96.805100,8.299783


In [29]:
display_experiment_results(experiment_results_true["cnn14_logstft"]["gradcam_clean"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.777619,21.794872,0.227320,0.698718,0.237156,31.810478,5.029264
topK_30_pos,0.594974,14.102564,0.279717,0.519231,0.464397,51.586828,6.695281
minmax_pos,0.337215,0.000000,0.000000,0.108974,0.618437,90.448146,9.652682
topK_5_pos,0.171583,4.166667,0.090463,0.121795,0.873857,88.453087,7.634905
pos_thresh_50,0.281354,0.320513,0.266586,0.128205,0.777310,88.980275,8.772010
topK_50,0.621108,3.205128,1.185983,0.490385,0.499945,56.917732,10.602242
pos_thresh_25,0.427502,2.243590,0.786154,0.314103,0.609550,71.119038,9.441787
bin,0.510466,7.051282,1.019737,0.560897,0.452052,49.886688,8.809827
minmax,0.596427,0.961538,0.059732,0.307692,0.302410,73.605895,11.132301
topK_5,0.112911,0.320513,0.089942,0.041667,0.949976,96.201768,8.299985


In [30]:
display_experiment_results(experiment_results_true["cnn14_logstft"]["lime_clean"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.819873,16.025641,8.791465,0.487179,0.477505,52.765375,10.275113
topK_30_pos,0.736013,9.935897,5.330596,0.378205,0.691351,64.301468,10.119489
minmax_pos,0.663173,0.320513,0.161831,0.195513,0.618076,82.921527,10.539677
topK_5_pos,0.240962,0.320513,0.047541,0.153846,0.936539,89.954697,8.527599
pos_thresh_50,0.439892,0.641026,0.345008,0.211538,0.865942,82.839096,9.122823
topK_50,0.819409,14.743590,8.993968,0.471154,0.493327,54.287029,10.615327
pos_thresh_25,0.718068,6.410256,2.899278,0.391026,0.694511,64.846598,9.999663
bin,0.852587,26.282051,14.748848,0.740385,0.263340,27.300729,10.937771
minmax,0.723457,1.923077,0.862809,0.269231,0.323994,75.558178,11.092808
topK_5,0.240962,0.320513,0.047541,0.153846,0.936539,89.954697,8.527599


In [31]:
display_experiment_results(experiment_results_true["cnn14_logstft"]["shap_clean"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.866932,3.525641,0.001048,0.128205,0.442302,86.934263,9.378820
topK_30_pos,0.819402,0.320513,0.247077,0.025641,0.699991,97.677030,10.091335
minmax_pos,0.494791,0.000000,0.000000,0.022436,0.733112,97.260683,10.237796
topK_5_pos,0.476380,0.000000,0.000000,0.012821,0.949986,97.279296,8.299783
pos_thresh_50,0.045245,0.641026,0.145455,0.009615,0.997592,97.715419,4.940646
topK_50,0.860424,0.000000,0.000000,0.019231,0.499994,97.549404,10.602145
pos_thresh_25,0.342238,0.000000,0.000000,0.022436,0.963501,97.027136,7.854634
bin,0.859379,0.000000,0.000000,0.019231,0.491929,97.497706,10.618033
minmax,0.817470,0.641026,0.473305,0.089744,0.072855,92.578307,11.285715
topK_5,0.476380,0.000000,0.000000,0.012821,0.949986,97.279296,8.299783


## STFT model clean - incorrect

In [32]:
display_experiment_results(experiment_results_false["cnn14_logstft"]["saliency_clean"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.583857,21.590909,0.002940,0.500000,0.249997,49.940538,5.301072
topK_30_pos,0.555864,3.409091,3.089041,0.034091,0.699991,96.081514,10.091335
minmax_pos,0.589870,4.545455,1.566937,0.079545,0.788733,90.544596,9.957932
topK_5_pos,0.512585,3.409091,2.632922,0.034091,0.949986,96.021290,8.299783
pos_thresh_50,0.053851,1.136364,0.681866,0.034091,0.998995,94.745940,3.573356
topK_50,0.573580,2.272727,1.959591,0.022727,0.499994,96.829311,10.602145
pos_thresh_25,0.262116,3.409091,0.675997,0.022727,0.991514,95.267087,6.229557
bin,0.573162,2.272727,1.958704,0.022727,0.500076,96.794247,10.601947
minmax,0.573092,15.909091,9.150131,0.238636,0.037301,68.956449,11.292049
topK_5,0.512585,3.409091,2.632922,0.034091,0.949986,96.021290,8.299783


In [33]:
display_experiment_results(experiment_results_false["cnn14_logstft"]["gradcam_clean"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.536529,35.227273,0.184548,0.715909,0.181783,28.467397,3.855392
topK_30_pos,0.492952,23.863636,0.004043,0.511364,0.365870,47.674545,5.275132
minmax_pos,0.260011,2.272727,0.398896,0.068182,0.580542,91.792949,8.261096
topK_5_pos,0.259980,13.636364,0.002665,0.284091,0.701678,69.958815,6.130854
pos_thresh_50,0.214227,0.000000,0.000000,0.079545,0.709466,90.265962,7.500912
topK_50,0.460792,6.818182,1.035443,0.250000,0.499912,70.300123,10.602308
pos_thresh_25,0.295724,2.272727,1.637856,0.102273,0.570356,83.870576,8.082806
bin,0.347414,9.090909,1.251668,0.272727,0.449259,69.129412,7.288249
minmax,0.464737,4.545455,2.156380,0.102273,0.270848,83.099057,11.163179
topK_5,0.121334,0.000000,0.000000,0.034091,0.949964,94.726891,8.300231


In [34]:
display_experiment_results(experiment_results_false["cnn14_logstft"]["lime_clean"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.571876,19.318182,8.291065,0.329545,0.454352,64.427788,9.770401
topK_30_pos,0.545752,10.227273,4.974083,0.193182,0.692452,77.441519,10.115883
minmax_pos,0.528683,3.409091,1.162153,0.125000,0.632914,89.202784,10.485179
topK_5_pos,0.282679,2.272727,0.446200,0.068182,0.936471,88.882856,8.528274
pos_thresh_50,0.402962,1.136364,0.396439,0.079545,0.876820,89.419504,9.023198
topK_50,0.569587,18.181818,9.447117,0.272727,0.493305,70.011761,10.615364
pos_thresh_25,0.529402,3.409091,1.766459,0.136364,0.705919,81.873358,9.938019
bin,0.569395,25.000000,12.278881,0.488636,0.305734,48.791411,10.909807
minmax,0.548853,6.818182,4.038918,0.125000,0.267736,84.486198,11.148491
topK_5,0.282679,2.272727,0.446200,0.068182,0.936471,88.882856,8.528274


In [35]:
display_experiment_results(experiment_results_false["cnn14_logstft"]["shap_clean"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.572406,4.545455,1.713856,0.079545,0.471585,91.785030,9.999750
topK_30_pos,0.567148,1.136364,0.883552,0.034091,0.699991,96.320111,10.091335
minmax_pos,0.315426,1.136364,0.120598,0.011364,0.732705,94.697038,10.239613
topK_5_pos,0.333723,1.136364,0.136690,0.034091,0.949986,93.894961,8.299783
pos_thresh_50,0.034279,1.136364,0.254741,0.011364,0.997548,96.957944,4.916781
topK_50,0.573036,2.272727,1.713665,0.022727,0.499994,97.463520,10.602145
pos_thresh_25,0.267677,0.000000,0.000000,0.011364,0.964013,96.971944,7.836621
bin,0.572843,2.272727,1.710430,0.022727,0.489782,97.454102,10.622260
minmax,0.547897,3.409091,1.015477,0.068182,0.072308,92.218731,11.285838
topK_5,0.333723,1.136364,0.136690,0.034091,0.949986,93.894961,8.299783


## Mask experiments clean - mel model

In [36]:
display_experiment_results(mask_experiment_results["cnn14_logmel"]["clean"])


--- topK_50_pos ---


,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.923509,17.000000,0.010820,0.660000,0.173750,34.228729,3.364614
gradcam,0.620931,11.750000,0.986515,0.652500,0.376861,38.971917,7.311441
lime,0.827086,13.500000,5.598555,0.750000,0.461311,30.218063,9.410670
shap,0.927128,0.000000,0.000000,0.017500,0.500000,98.489136,9.682342



--- topK_30_pos ---


,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.924342,0.000000,0.000000,0.015000,0.700006,98.380550,9.171495
gradcam,0.362777,8.000000,0.610826,0.445000,0.594143,60.972215,7.798626
lime,0.623726,10.000000,4.870080,0.542500,0.674179,49.493281,9.251675
shap,0.920323,0.000000,0.000000,0.015000,0.700006,98.376693,9.171495



--- minmax_pos ---


...

,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.494250,3.500000,1.586628,0.705000,0.000435,38.663598,10.375488
gradcam,0.503142,2.000000,0.972570,0.677500,0.026291,41.579207,10.373997
lime,0.497470,2.000000,0.954793,0.677500,0.014309,42.379379,10.374871
shap,0.411541,1.750000,0.706020,0.635000,0.001600,45.859315,10.375483



--- sigmoid_pos ---


,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.459911,2.500000,1.171660,0.677500,0.000218,41.395925,10.375489
gradcam,0.509011,2.750000,1.287992,0.672500,0.021327,41.113377,9.803656
lime,0.496602,2.000000,0.963045,0.675000,0.013037,42.468471,10.374927
shap,0.413540,2.000000,0.717767,0.632500,0.000816,45.765673,10.375487



--- pos_thresh_75 ---


,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.004156,0.000000,0.000000,0.030000,0.998694,97.389923,1.330769
gradcam,0.158799,0.250000,0.219627,0.220000,0.782398,83.862255,7.872705
lime,0.238406,3.250000,1.655848,0.350000,0.861821,68.189178,8.186731
shap,0.000941,0.000000,0.000000,0.027500,0.998980,97.767977,1.983555


## Mask experiments - stft model

In [37]:
display_experiment_results(mask_experiment_results["cnn14_logstft"]["clean"])


--- topK_50_pos ---


,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.804243,21.250000,0.005357,0.515000,0.246247,48.376074,5.221556
gradcam,0.724579,24.750000,0.217910,0.702500,0.224974,31.075000,4.771012
lime,0.765314,16.750000,8.681377,0.452500,0.472411,55.331106,10.164076
shap,0.802137,3.750000,0.377866,0.117500,0.448744,88.001432,9.515425



--- topK_30_pos ---


,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.775971,1.000000,0.695137,0.030000,0.699991,96.617813,10.091335
gradcam,0.572529,16.250000,0.219069,0.517500,0.442721,50.726126,6.382848
lime,0.694155,10.000000,5.252163,0.337500,0.691593,67.192280,10.118696
shap,0.763906,0.500000,0.387102,0.027500,0.699991,97.378508,10.091335



--- minmax_pos ---


...

,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.643469,2.500000,0.838001,0.202500,0.000142,79.326670,11.295279
gradcam,0.634957,1.750000,0.747174,0.190000,0.014194,80.017202,11.294890
lime,0.647775,2.500000,0.875570,0.217500,0.007533,78.297433,11.295110
shap,0.638561,2.000000,0.676899,0.190000,0.000569,79.908981,11.295279



--- sigmoid_pos ---


,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.641400,2.000000,0.750599,0.197500,0.000071,79.565167,11.295279
gradcam,0.645253,2.250000,0.778206,0.205000,0.007431,79.146000,10.504438
lime,0.648529,2.500000,0.878564,0.222500,0.006348,78.271001,11.295134
shap,0.638853,2.000000,0.679504,0.192500,0.000291,79.869016,11.295279



--- pos_thresh_75 ---


,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.006663,0.000000,0.000000,0.010000,0.997741,97.268040,1.625156
gradcam,0.101793,0.000000,0.000000,0.050000,0.879805,95.176441,7.329255
lime,0.213145,0.250000,0.078971,0.122500,0.943129,90.821143,8.265318
shap,0.000926,0.250000,0.015382,0.012500,0.998292,97.660085,2.156133


___

## Noisy experiments - mel model

In [38]:
display_experiment_results(experiment_results["cnn14_logmel"]["saliency_white"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.711190,32.500000,0.009746,0.745000,0.127500,25.342996,2.468997
topK_30_pos,0.713284,0.500000,0.175175,0.022500,0.700006,97.727077,9.171495
minmax_pos,0.692917,0.500000,0.219268,0.100000,0.783581,92.279278,9.085023
topK_5_pos,0.684098,0.000000,0.000000,0.007500,0.949973,98.185153,7.380256
pos_thresh_50,0.124662,0.500000,0.232667,0.030000,0.998953,96.182471,2.896074
topK_50,0.712000,0.500000,0.149811,0.022500,0.500000,97.723036,9.682342
pos_thresh_25,0.401284,0.500000,0.384639,0.017500,0.992539,97.143905,5.222684
bin,0.711829,0.500000,0.155864,0.022500,0.502705,97.702287,9.676883
minmax,0.696012,4.750000,3.127126,0.102500,0.035328,89.096607,10.372436
topK_5,0.684098,0.000000,0.000000,0.007500,0.949973,98.185153,7.380256


In [39]:
display_experiment_results(experiment_results["cnn14_logmel"]["saliency_room"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.850689,27.250000,0.010742,0.685000,0.157500,31.044639,3.049938
topK_30_pos,0.850516,0.000000,0.000000,0.012500,0.700006,98.061626,9.171495
minmax_pos,0.792496,0.500000,0.211552,0.082500,0.776968,94.060062,9.124363
topK_5_pos,0.766804,0.000000,0.000000,0.020000,0.949973,97.426624,7.380256
pos_thresh_50,0.084152,0.500000,0.212758,0.017500,0.998847,97.245856,3.084981
topK_50,0.850997,0.000000,0.000000,0.012500,0.500000,98.094113,9.682342
pos_thresh_25,0.387530,0.750000,0.328547,0.025000,0.990762,96.975251,5.462471
bin,0.851018,0.000000,0.000000,0.012500,0.501807,98.102482,9.678695
minmax,0.824637,3.250000,2.295471,0.117500,0.038908,88.139321,10.371981
topK_5,0.766804,0.000000,0.000000,0.020000,0.949973,97.426624,7.380256


In [40]:
display_experiment_results(experiment_results["cnn14_logmel"]["saliency_horse"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.790323,19.250000,0.007534,0.490000,0.256250,49.701692,4.962200
topK_30_pos,0.788420,0.000000,0.000000,0.002500,0.700006,97.230655,9.171495
minmax_pos,0.769024,0.250000,0.228765,0.067500,0.774734,92.016089,9.143446
topK_5_pos,0.695200,0.500000,0.155136,0.012500,0.949973,95.898839,7.380256
pos_thresh_50,0.107027,0.500000,0.203184,0.010000,0.998757,97.487749,3.272373
topK_50,0.789539,0.000000,0.000000,0.002500,0.500000,96.983300,9.682342
pos_thresh_25,0.386120,1.000000,0.463934,0.007500,0.989099,97.536813,5.682875
bin,0.789483,0.000000,0.000000,0.002500,0.499990,96.975555,9.682347
minmax,0.774518,10.750000,7.401402,0.277500,0.040483,72.553143,10.371809
topK_5,0.695200,0.500000,0.155136,0.012500,0.949973,95.898839,7.380256


In [41]:
display_experiment_results(experiment_results["cnn14_logmel"]["gradcam_white"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.614212,22.750000,0.895524,0.607500,0.279518,41.201800,5.423072
topK_30_pos,0.507756,17.750000,0.732232,0.435000,0.470052,56.851181,6.170159
minmax_pos,0.410126,1.750000,0.942711,0.162500,0.455897,86.231142,8.169024
topK_5_pos,0.216792,9.750000,0.001232,0.220000,0.785244,78.910743,6.123868
pos_thresh_50,0.367672,6.000000,1.895025,0.245000,0.578542,77.531645,7.416078
topK_50,0.552914,5.500000,2.520131,0.292500,0.499065,72.616278,9.684207
pos_thresh_25,0.469095,9.500000,2.522458,0.367500,0.425811,64.546311,6.748737
bin,0.526246,15.750000,2.547937,0.510000,0.291736,49.573783,5.396175
minmax,0.545293,1.750000,1.096807,0.205000,0.346065,81.683006,10.148230
topK_5,0.134890,0.250000,0.061107,0.070000,0.948927,94.279909,7.400621


In [42]:
display_experiment_results(experiment_results["cnn14_logmel"]["gradcam_room"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.668329,15.500000,1.049485,0.617500,0.328206,42.246285,6.367224
topK_30_pos,0.487269,10.000000,0.239408,0.420000,0.531187,61.041883,6.973047
minmax_pos,0.423070,0.500000,0.199785,0.227500,0.443758,82.149382,8.831108
topK_5_pos,0.146072,4.500000,0.001316,0.157500,0.846876,85.622335,6.605853
pos_thresh_50,0.360350,2.750000,0.839456,0.307500,0.578900,74.603804,8.099933
topK_50,0.595938,4.500000,2.336654,0.417500,0.499191,62.563061,9.683955
pos_thresh_25,0.527204,10.500000,3.708322,0.525000,0.398001,52.277242,7.448694
bin,0.624585,16.750000,2.458265,0.645000,0.262752,37.184366,5.513788
minmax,0.553362,1.250000,0.568341,0.272500,0.331609,76.072015,10.168424
topK_5,0.077781,0.000000,0.000000,0.062500,0.948867,95.579716,7.401766


In [43]:
display_experiment_results(experiment_results["cnn14_logmel"]["gradcam_horse"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.668687,16.750000,0.851790,0.522500,0.343045,50.511042,6.658016
topK_30_pos,0.456246,8.750000,0.002277,0.285000,0.576740,73.302948,7.569022
minmax_pos,0.503738,0.750000,0.092385,0.147500,0.489186,88.137892,9.195824
topK_5_pos,0.118517,3.750000,0.000537,0.100000,0.884974,90.037609,6.899170
pos_thresh_50,0.449253,2.000000,0.546866,0.235000,0.606197,80.000784,8.682833
topK_50,0.612718,3.750000,1.824735,0.307500,0.499043,71.618280,9.684251
pos_thresh_25,0.575466,7.750000,1.807158,0.435000,0.437871,59.357193,8.634937
bin,0.646846,13.750000,2.481985,0.595000,0.311681,42.425838,6.834141
minmax,0.665791,2.250000,0.614114,0.255000,0.302349,78.446919,10.195314
topK_5,0.093737,0.250000,0.023225,0.037500,0.948990,96.102285,7.399404


In [44]:
display_experiment_results(experiment_results["cnn14_logmel"]["lime_white"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.661303,15.500000,5.588439,0.500000,0.436843,50.420955,9.111431
topK_30_pos,0.579218,10.250000,4.118558,0.360000,0.652824,64.583169,9.145879
minmax_pos,0.640144,5.500000,2.944553,0.292500,0.518474,72.856361,9.778440
topK_5_pos,0.287504,2.500000,0.941906,0.165000,0.905188,83.750265,7.976897
pos_thresh_50,0.543765,8.250000,4.268772,0.325000,0.710520,67.540012,8.939172
topK_50,0.662237,13.750000,6.133981,0.472500,0.466945,53.109298,9.745318
pos_thresh_25,0.655523,17.500000,8.751906,0.555000,0.506051,44.673501,9.432342
bin,0.698812,34.250000,9.174560,0.802500,0.187839,19.663277,8.035213
minmax,0.650651,7.500000,3.858202,0.312500,0.404169,70.803854,10.041012
topK_5,0.287504,2.500000,0.941906,0.165000,0.905188,83.750265,7.976897


In [45]:
display_experiment_results(experiment_results["cnn14_logmel"]["lime_room"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.798876,11.000000,3.884707,0.525000,0.465679,49.757447,9.482351
topK_30_pos,0.676907,6.000000,2.684843,0.387500,0.673884,64.779027,9.252643
minmax_pos,0.756607,2.000000,0.992042,0.267500,0.571788,78.139046,9.702122
topK_5_pos,0.266320,2.000000,1.103010,0.170000,0.917428,84.912728,7.843242
pos_thresh_50,0.545555,5.000000,2.517660,0.302500,0.779319,73.168008,8.676935
topK_50,0.799205,9.750000,4.356550,0.515000,0.477792,50.985642,9.725162
pos_thresh_25,0.765974,10.750000,4.329350,0.495000,0.599181,53.205151,9.282798
bin,0.834633,22.750000,10.066187,0.822500,0.209662,21.224098,9.733605
minmax,0.772891,2.500000,0.969162,0.275000,0.402839,75.501471,10.055631
topK_5,0.266320,2.000000,1.103010,0.170000,0.917428,84.912728,7.843242


In [46]:
display_experiment_results(experiment_results["cnn14_logmel"]["lime_horse"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.764370,19.750000,9.932483,0.655000,0.439142,38.065049,8.974500
topK_30_pos,0.662887,14.750000,8.590946,0.405000,0.672999,59.581706,9.227538
minmax_pos,0.699502,6.750000,4.104590,0.262500,0.604543,75.048172,9.633056
topK_5_pos,0.273862,4.500000,2.608519,0.205000,0.916421,82.605794,7.849365
pos_thresh_50,0.527689,10.250000,6.915780,0.317500,0.789854,70.807660,8.659089
topK_50,0.760747,21.750000,12.180873,0.640000,0.476016,39.704406,9.728505
pos_thresh_25,0.703865,15.500000,10.398968,0.512500,0.627666,50.686515,9.279216
bin,0.780193,40.000000,20.360500,0.827500,0.291061,16.939316,9.829493
minmax,0.730194,10.250000,6.102790,0.382500,0.368780,65.278803,10.106139
topK_5,0.273862,4.500000,2.608519,0.205000,0.916421,82.605794,7.849365


In [47]:
display_experiment_results(experiment_results["cnn14_logmel"]["shap_white"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.711890,0.750000,0.244811,0.022500,0.500000,97.825080,9.682342
topK_30_pos,0.712049,0.750000,0.239593,0.022500,0.700006,97.841733,9.171495
minmax_pos,0.703279,0.250000,0.133590,0.012500,0.721409,97.694101,9.358230
topK_5_pos,0.697979,0.250000,0.142631,0.010000,0.949973,97.190329,7.380256
pos_thresh_50,0.153013,1.000000,0.414723,0.037500,0.996068,96.786666,4.462922
topK_50,0.711890,0.750000,0.244811,0.022500,0.500000,97.825080,9.682342
pos_thresh_25,0.672607,0.000000,0.000000,0.007500,0.949855,97.280186,7.272439
bin,0.711994,0.750000,0.248285,0.022500,0.488620,97.833374,9.704824
minmax,0.707674,0.000000,0.000000,0.017500,0.081260,97.210558,10.363955
topK_5,0.697979,0.250000,0.142631,0.010000,0.949973,97.190329,7.380256


In [48]:
display_experiment_results(experiment_results["cnn14_logmel"]["shap_room"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.851707,0.000000,0.000000,0.012500,0.500000,98.118999,9.682342
topK_30_pos,0.848558,0.000000,0.000000,0.012500,0.700006,98.169867,9.171495
minmax_pos,0.827403,0.000000,0.000000,0.015000,0.721625,98.417283,9.356942
topK_5_pos,0.824963,0.000000,0.000000,0.020000,0.949973,97.573005,7.380256
pos_thresh_50,0.094349,0.000000,0.000000,0.015000,0.996874,97.482737,4.248237
topK_50,0.851707,0.000000,0.000000,0.012500,0.500000,98.118999,9.682342
pos_thresh_25,0.736986,0.000000,0.000000,0.002500,0.957153,98.339161,7.097326
bin,0.851762,0.000000,0.000000,0.012500,0.486988,98.107902,9.707996
minmax,0.844402,0.000000,0.000000,0.032500,0.066837,97.179150,10.367339
topK_5,0.824963,0.000000,0.000000,0.020000,0.949973,97.573005,7.380256


In [49]:
display_experiment_results(experiment_results["cnn14_logmel"]["shap_horse"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.790697,0.250000,0.000001,0.005000,0.498750,96.701207,9.658136
topK_30_pos,0.783312,0.000000,0.000000,0.005000,0.700006,97.136149,9.171495
minmax_pos,0.753468,0.000000,0.000000,0.007500,0.727941,97.820285,9.333287
topK_5_pos,0.731777,0.500000,0.035973,0.022500,0.949973,95.778591,7.380256
pos_thresh_50,0.105702,0.500000,0.183802,0.017500,0.996972,97.933303,4.249547
topK_50,0.790343,0.000000,0.000000,0.002500,0.500000,96.830049,9.682342
pos_thresh_25,0.566562,0.250000,0.056329,0.025000,0.961684,96.388800,6.980273
bin,0.790175,0.000000,0.000000,0.005000,0.488257,96.767103,9.705524
minmax,0.788614,0.500000,0.111171,0.017500,0.067695,97.039444,10.367169
topK_5,0.731777,0.500000,0.035973,0.022500,0.949973,95.778591,7.380256


## Noisy experiments per mask - mel model

In [50]:
display_experiment_results(mask_experiment_results["cnn14_logmel"]["white"])


--- topK_50_pos ---


,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.711190,32.500000,0.009746,0.745000,0.127500,25.342996,2.468997
gradcam,0.614212,22.750000,0.895524,0.607500,0.279518,41.201800,5.423072
lime,0.661303,15.500000,5.588439,0.500000,0.436843,50.420955,9.111431
shap,0.711890,0.750000,0.244811,0.022500,0.500000,97.825080,9.682342



--- topK_30_pos ---


,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.713284,0.500000,0.175175,0.022500,0.700006,97.727077,9.171495
gradcam,0.507756,17.750000,0.732232,0.435000,0.470052,56.851181,6.170159
lime,0.579218,10.250000,4.118558,0.360000,0.652824,64.583169,9.145879
shap,0.712049,0.750000,0.239593,0.022500,0.700006,97.841733,9.171495



--- minmax_pos ---


...

,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.551320,8.500000,4.458095,0.475000,0.000461,59.530889,10.375488
gradcam,0.514549,5.250000,1.910096,0.372500,0.021900,67.793831,10.374392
lime,0.557132,8.000000,3.849045,0.400000,0.015988,65.894752,10.374765
shap,0.479692,5.000000,1.995513,0.380000,0.001059,69.050048,10.375487



--- sigmoid_pos ---


,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.522607,7.750000,3.453662,0.437500,0.000231,62.839514,10.375489
gradcam,0.523618,5.500000,2.506272,0.380000,0.014240,66.116355,8.922229
lime,0.555828,7.750000,3.768415,0.397500,0.014469,65.821766,10.374838
shap,0.479328,5.000000,1.980919,0.382500,0.000543,68.713899,10.375488



--- pos_thresh_75 ---


,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.036817,0.000000,0.000000,0.037500,0.998527,96.274690,1.169532
gradcam,0.221480,0.500000,0.139204,0.102500,0.744058,91.551230,6.855847
lime,0.408778,5.250000,2.258611,0.232500,0.833223,77.525128,8.406193
shap,0.014333,0.000000,0.000000,0.035000,0.998922,96.935569,1.893806


In [51]:
display_experiment_results(mask_experiment_results["cnn14_logmel"]["room"])


--- topK_50_pos ---


,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.850689,27.250000,0.010742,0.685000,0.157500,31.044639,3.049938
gradcam,0.668329,15.500000,1.049485,0.617500,0.328206,42.246285,6.367224
lime,0.798876,11.000000,3.884707,0.525000,0.465679,49.757447,9.482351
shap,0.851707,0.000000,0.000000,0.012500,0.500000,98.118999,9.682342



--- topK_30_pos ---


,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.850516,0.000000,0.000000,0.012500,0.700006,98.061626,9.171495
gradcam,0.487269,10.000000,0.239408,0.420000,0.531187,61.041883,6.973047
lime,0.676907,6.000000,2.684843,0.387500,0.673884,64.779027,9.252643
shap,0.848558,0.000000,0.000000,0.012500,0.700006,98.169867,9.171495



--- minmax_pos ---


...

,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.596879,6.000000,3.488200,0.572500,0.000517,51.145902,10.375488
gradcam,0.558418,3.000000,1.595462,0.535000,0.023324,57.186894,10.374343
lime,0.598701,3.250000,1.934571,0.502500,0.015792,56.989645,10.374776
shap,0.506164,3.000000,1.742738,0.487500,0.001007,61.328933,10.375487



--- sigmoid_pos ---


,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.560616,4.750000,2.694025,0.545000,0.000259,54.838752,10.375489
gradcam,0.565542,3.500000,1.996931,0.542500,0.016902,56.205008,9.492769
lime,0.596822,3.250000,1.912979,0.505000,0.014285,57.145403,10.374840
shap,0.507642,3.250000,1.760654,0.487500,0.000519,61.117844,10.375488



--- pos_thresh_75 ---


,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.023657,0.000000,0.000000,0.025000,0.998631,97.021990,1.284587
gradcam,0.182597,0.000000,0.000000,0.135000,0.779157,89.195469,7.443469
lime,0.331308,2.250000,1.468846,0.195000,0.880057,81.639244,8.071825
shap,0.002502,0.000000,0.000000,0.017500,0.998952,97.195290,1.867501


In [52]:
display_experiment_results(mask_experiment_results["cnn14_logmel"]["horse"])


--- topK_50_pos ---


,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.790323,19.250000,0.007534,0.490000,0.256250,49.701692,4.962200
gradcam,0.668687,16.750000,0.851790,0.522500,0.343045,50.511042,6.658016
lime,0.764370,19.750000,9.932483,0.655000,0.439142,38.065049,8.974500
shap,0.790697,0.250000,0.000001,0.005000,0.498750,96.701207,9.658136



--- topK_30_pos ---


,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.788420,0.000000,0.000000,0.002500,0.700006,97.230655,9.171495
gradcam,0.456246,8.750000,0.002277,0.285000,0.576740,73.302948,7.569022
lime,0.662887,14.750000,8.590946,0.405000,0.672999,59.581706,9.227538
shap,0.783312,0.000000,0.000000,0.005000,0.700006,97.136149,9.171495



--- minmax_pos ---


...

,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.611014,3.500000,1.996095,0.555000,0.000539,56.130944,10.375488
gradcam,0.586694,1.000000,0.272882,0.445000,0.045033,66.063212,10.371287
lime,0.600107,4.000000,1.709758,0.497500,0.018723,61.679100,10.374649
shap,0.524772,2.000000,0.652221,0.415000,0.001090,68.324628,10.375487



--- sigmoid_pos ---


,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.575920,3.000000,1.416806,0.512500,0.000269,61.363719,10.375489
gradcam,0.594669,2.250000,0.595393,0.470000,0.024390,63.589368,9.907195
lime,0.599394,2.750000,1.428018,0.480000,0.015101,62.498017,10.374842
shap,0.526666,2.000000,0.657394,0.422500,0.000559,68.180132,10.375488



--- pos_thresh_75 ---


,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.021554,0.000000,0.000000,0.012500,0.998796,96.980906,1.429625
gradcam,0.251565,0.500000,0.216440,0.075000,0.799444,92.968666,7.877564
lime,0.337981,5.250000,3.795244,0.232500,0.889431,80.521404,8.004952
shap,0.003826,0.000000,0.000000,0.020000,0.998917,97.433498,1.839846


## Summary tables

In [53]:
def _invert_metrics(df, invert_cols=("COMP","AD")):
    df = df.copy()
    for c in invert_cols:
        if c in df.columns:
            df[c] = - df[c]
    return df

def _metric_columns(df):
    return [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]

def build_method_exp_index(experiment_results_dict):
    index = defaultdict(lambda: defaultdict(dict))
    for model, exps in experiment_results_dict.items():
        for exp_name, df in exps.items():
            if "_" in exp_name:
                method, exp = exp_name.split("_", 1)
            else:
                method, exp = exp_name, "exp"
            index[model][exp][method] = df
    return index

def summarize_best_masks_by_experiment(experiment_results_dict,
                                       invert_cols=("COMP","AD"),
                                       known_methods=None):
    idx = build_method_exp_index(experiment_results_dict)
    summaries = defaultdict(dict)

    for model, exp_dict in idx.items():
        for exp, methods in exp_dict.items():
            rows = []
      
            method_list = list(methods.keys())
            if known_methods:
    
                method_list = [m for m in known_methods if m in methods]

            for m in method_list:
                df = methods[m]
    
                metrics = _metric_columns(df)
                if not metrics:
                    continue
                dfm = df[metrics].copy()
                dfm_inv = _invert_metrics(dfm, invert_cols=invert_cols)
                mean_score = dfm_inv.mean(axis=1)
                best_idx = mean_score.idxmax()
                out = {
                    "method": m,
                    "best_mask": best_idx,
                    "mean_score": float(mean_score.loc[best_idx]),
                }
   
                for col in metrics:
                    out[col] = float(dfm.loc[best_idx, col])
                rows.append(out)

            out_df = pd.DataFrame(rows).sort_values("mean_score", ascending=False, ignore_index=True)
            summaries[model][exp] = out_df
    return summaries

_source = experiment_results

summaries_v2 = summarize_best_masks_by_experiment(
    _source,
    invert_cols=("COMP","AD"),
    known_methods=method_names
)

for model, exps in summaries_v2.items():
    print(f"\n==================== MODEL: {model} ====================")
    for exp, df in exps.items():
        #print(f"\n--- Experiment: {exp} ---")
        #df = df[["method", "FF", "AI", "AG", "FidIn", "SPS", "AD", "COMP"]]
        #display(df)
        display_summary_table(df, title=f"--- Experiment: {exp} ---")


==================== MODEL: cnn14_logstft ====================

=== --- Experiment: clean --- ===


,FF,AI,AG,FidIn,SPS,AD,COMP
method,,,,,,,
saliency,0.804243,21.250000,0.005357,0.515000,0.246247,48.376074,5.221556
gradcam,0.724579,24.750000,0.217910,0.702500,0.224974,31.075000,4.771012
lime,0.790285,26.000000,14.205455,0.685000,0.272667,32.028679,10.931619
shap,0.638853,2.000000,0.679504,0.192500,0.000291,79.869016,11.295279



==================== MODEL: cnn14_logmel ====================

=== --- Experiment: clean --- ===


,FF,AI,AG,FidIn,SPS,AD,COMP
method,,,,,,,
saliency,0.923509,17.000000,0.010820,0.660000,0.173750,34.228729,3.364614
gradcam,0.718901,19.500000,2.312055,0.782500,0.195505,24.363625,4.918478
lime,0.899183,28.750000,12.359402,0.932500,0.200351,8.067020,9.534232
shap,0.413540,2.000000,0.717767,0.632500,0.000816,45.765673,10.375487



=== --- Experiment: white --- ===


,FF,AI,AG,FidIn,SPS,AD,COMP
method,,,,,,,
saliency,0.711190,32.500000,0.009746,0.745000,0.127500,25.342996,2.468997
gradcam,0.614212,22.750000,0.895524,0.607500,0.279518,41.201800,5.423072
lime,0.698812,34.250000,9.174560,0.802500,0.187839,19.663277,8.035213
shap,0.479328,5.000000,1.980919,0.382500,0.000543,68.713899,10.375488



=== --- Experiment: room --- ===


,FF,AI,AG,FidIn,SPS,AD,COMP
method,,,,,,,
saliency,0.850689,27.250000,0.010742,0.685000,0.157500,31.044639,3.049938
gradcam,0.624585,16.750000,2.458265,0.645000,0.262752,37.184366,5.513788
lime,0.834633,22.750000,10.066187,0.822500,0.209662,21.224098,9.733605
shap,0.507642,3.250000,1.760654,0.487500,0.000519,61.117844,10.375488



=== --- Experiment: horse --- ===


,FF,AI,AG,FidIn,SPS,AD,COMP
method,,,,,,,
saliency,0.790323,19.250000,0.007534,0.490000,0.256250,49.701692,4.962200
gradcam,0.646846,13.750000,2.481985,0.595000,0.311681,42.425838,6.834141
lime,0.780193,40.000000,20.360500,0.827500,0.291061,16.939316,9.829493
shap,0.526666,2.000000,0.657394,0.422500,0.000559,68.180132,10.375488


## Saving results

In [54]:
os.makedirs("supplementary", exist_ok=True)
output_dir = os.path.join("supplementary", "exported_csvs")
os.makedirs(output_dir, exist_ok=True)

for model_type, exps in experiment_results.items():
    for exp_name, df in exps.items():
        fname = f"{model_type}_{exp_name}.csv"
        df.to_csv(os.path.join(output_dir, fname), index=True)

for model_type, exps in mask_experiment_results.items():
    for exp_type, masks in exps.items():
        for mask_name, df in masks.items():
            fname = f"{model_type}_{exp_type}_{mask_name}.csv"
            df.to_csv(os.path.join(output_dir, fname), index=True)

path_exported_true = os.path.join("supplementary", "exported_true")
path_exported_false = os.path.join("supplementary", "exported_false")
os.makedirs(path_exported_true, exist_ok=True)
os.makedirs(path_exported_false, exist_ok=True)

for model in model_types:
    for exp_name, df_t in experiment_results_true[model].items():
        df_t.to_csv(os.path.join(path_exported_true, f"{model}_{exp_name}_true.csv"), index=True)
    for exp_name, df_f in experiment_results_false[model].items():
        df_f.to_csv(os.path.join(path_exported_false, f"{model}_{exp_name}_false.csv"), index=True)